In [3]:
import numpy as np 
import random
from functions import *

In [ ]:
data_tpts, data_x, data_y = data_deal('Sethylene%')

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import random
from functions_new import *

def MBR_xgboost_func(x):
    n_basis, order, eta, max_depth, min_child_weight = x
    n_basis = int(n_basis)
    order = int(order)
    max_depth = int(max_depth)
    min_child_weight = int(min_child_weight)
    
    if n_basis <= order:
        return 1000000
    model_params = {'eta': eta,
                    'max_depth':max_depth,
                    "min_child_weight":min_child_weight,
                    'objective': 'reg:squarederror'}
    n = 5
    random.seed(1)
    seed = list(random.sample(list(range(1, 1001)), n))
    R2_mx = []
    mise_mx = []
    y_pred_to_mx = []
    test_idx_to_mx = []
    for i in range(n):
        out_put_mx = FBboost(data_x, data_y, model_params, 
                            data_tpts, n_basis=n_basis, order=order, 
                            num_rounds=100,loss_type=1,test_size=0.2,
                            seed=seed,ite=i,silent=True)
        R2_mx.append(out_put_mx['R2_score'])
        mise_mx.append(out_put_mx['mise_score'])
        y_pred_to_mx.append(out_put_mx['y_pred'])
        test_idx_to_mx.append(out_put_mx['test_idx'])
    print(f'{x}')
    print(f'The mise is {np.mean(mise_mx)}')
    print(f'The R2 is {np.mean(R2_mx)}')
    print('\n')
    
    return np.mean(mise_mx)

# dragonfly

In [ ]:
from dragonfly import load_config_file, minimise_function
from argparse import Namespace

""" Main function. """
config = load_config_file("dragonfly_config.json")
max_capital = 60
options = Namespace(
    build_new_model_every=5, # update the model every 5 iterations
    report_results_every=4, # report progress every 4 iterations
    report_model_on_each_build=True, # report model when you build it.
    )
opt_method = 'bo'
opt_val, opt_pt, history = minimise_function(MBR_xgboost_func,
                                             config.domain,
                                             max_capital,
                                             opt_method=opt_method,
                                             config=config,
                                             options=options
                                            )
print('Optimum Value in %d evals: %0.4f'%(max_capital, opt_val))
print('Optimum Point: %s'%(opt_pt))